# Data Wrangling Final Project: Purple Air

**Purple Air**

PurpleAir sensors employ a dual laser counter to provide some level of data integrity. This is intended to provide a way of determining sensor health and fault detection. Some examples of what can go wrong with a laser counter are a fan failure, insects or other debris inside the device or just a layer of dust from long term exposure. If both laser counters (channels) are in agreement, the data can be seen as excellent quality. If there are different readings from the two channels, there may be a fault with one or both.In the case of a fault, the channel may be marked as flagged or downgraded (suspect or known faulty).

Two API Variants:
1. JSON Legacy
2. JSON Experimental
3. (Private sensors are not querried. Can ask for permission if you want!)

## Data Collection:

In [3]:
#Libaries
#Use the lab_basic venv

#Basics:
import pandas as pd
import matplotlib.pyplot as plt
import os

#Scraping:
import requests
import json
from urllib.request import urlopen,urlretrieve
import sys
from datetime import datetime
import pytz


#Prediction
from fbprophet import Prophet
import statsmodels.formula.api as smf
import statsmodels.api as sm

#Geography
import geopandas as gpd
import fiona
from shapely.geometry import Point
import descartes
import contextily as ctx #Basemaps 

# Collect Data

### PurpleAir Data

Data collection for purpleair data is being run through their json querry and an airflow scheduling assistant. Essentially, purpleair only posts realtime data from their sensors to the web at an unknown refresh rate. There is no way to querry data from the past. Instead, one must manually collect scrape the data at some frequency for analysis.

To this end, I've put together an airflow DAG to collect this data, arange it into a pandas dataframe and pickle it for further use. This data would be best put into a SQL database. To ensure this project maintained a manageabler workload, I decided to take on only one new technology (airflow). Further down the road, I would like to create automate the datastream into a database to minimize footprint. 

To see the DAG file, please look at the .py file contained within this folder. 

All data is saved as either legacy/experimental _MONTH_DAY_HOUR_MIN .pkl

**Let's look at a subsample of the data**

In [9]:
directory = '/home/boogie2/external/tiny_tower_2/purpleair_data/'
data_direct = []

#Create a directory of the data
for filename in os.listdir(directory):
    
    #append the path to the list
    data_direct.append(os.path.join(directory, filename))
    
    #sort the data
    data_direct = sorted(data_direct)

In [10]:
print('There are currently {0} entries in the data folder'.format(len(data_direct)))

There are currently 29 entries in the data folder


In [14]:
#Open one a datafile
print(data_direct[-2])

print(data_direct[13])

/home/boogie2/external/tiny_tower_2/purpleair_data/legacy_04_07_18_00.pkl
/home/boogie2/external/tiny_tower_2/purpleair_data/experimental_04_07_18_00.pkl


In [16]:
#Let's open a legacy test file
leg_test = pd.read_pickle(data_direct[-2])
leg_test.head()

,mapVersion,baseVersion,mapVersionString,ID,Label,DEVICE_LOCATIONTYPE,THINGSPEAK_PRIMARY_ID,THINGSPEAK_PRIMARY_ID_READ_KEY,THINGSPEAK_SECONDARY_ID,THINGSPEAK_SECONDARY_ID_READ_KEY,...,temp_f,pressure,AGE,Stats,ParentID,Flag,A_H,Voc,Ozone1,THINGSPEAK_SEY_ID
0,0.4,7,,14633,Hazelwood canary,outside,559921,CU4BQZZ38WO5UJ4C,559922,D0YNZ1LM59LL49VQ,...,76,1010.33,0,"{""v"":0.9,""v1"":0.68,""v2"":0.87,""v3"":0.92,""v4"":1....",NaN,NaN,NaN,NaN,NaN,NaN
1,0.4,7,,14634,Hazelwood canary B,NaN,559923,DULWDNCI9M6PCIPC,559924,EY2CNMYRUZHDW1AL,...,NaN,NaN,0,"{""v"":0.0,""v1"":0.05,""v2"":0.15,""v3"":0.17,""v4"":0....",14633.0,1.0,NaN,NaN,NaN,NaN
2,0.4,7,,25999,Villages of Bridgestone AQI,outside,694803,OO5PFS7JTQQSHQHE,694804,LW93B1YLLZ4N4QFM,...,93,1008.43,1,"{""v"":28.47,""v1"":27.7,""v2"":27.43,""v3"":27.09,""v4...",NaN,1.0,NaN,NaN,NaN,NaN
3,0.4,7,,26000,Villages of Bridgestone AQI B,NaN,694805,OAPUW1RM0X9CJDYO,694806,XAFZ75MBJ90SZIU7,...,NaN,NaN,1,"{""v"":26.09,""v1"":25.43,""v2"":25.09,""v3"":24.78,""v...",25999.0,NaN,NaN,NaN,NaN,NaN
4,0.4,7,,16931,#SFAQ10,outside,592265,1ENJG2P25X9OVDRN,592267,8QT4N8YSSJ8M2LVD,...,81,984.96,0,"{""v"":3.66,""v1"":2.72,""v2"":2.64,""v3"":3.39,""v4"":8...",NaN,1.0,NaN,NaN,NaN,NaN


In [17]:
#Let's open a experimental data file
exp_test = pd.read_pickle(data_direct[13])
exp_test.head()

,ID,pm,age,pm_0,pm_1,pm_2,pm_3,pm_4,pm_5,pm_6,...,Lon,Icon,isOwner,Flags,Voc,Ozone1,Adc,CH,version,count
0,20,1.0,1,1.0,0.9,0.8,0.7,0.6,0.7,1.3,...,-111.836120,0,0,0,NaN,NaN,0.01,1,7.0.4,7254
1,47,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-111.882600,0,0,0,NaN,1.83,1.83,0,7.0.4,7254
2,53,1.0,0,1.0,1.1,1.5,3.1,13.4,13.5,9.3,...,-111.704800,0,0,0,NaN,NaN,0.00,1,7.0.4,7254
3,72,18.2,0,18.2,18.6,19.0,19.5,21.3,22.9,25.5,...,-111.947075,0,0,0,NaN,NaN,0.05,1,7.0.4,7254
4,77,4.1,0,4.1,4.3,4.4,4.3,4.6,5.1,6.3,...,-111.825290,0,0,0,NaN,NaN,0.01,1,7.0.4,7254


**Conclusion**
The data collection appears to be working. Consistent recordings start at 04/07/20 @ 5:00 AM (MST)

### EPA Airnow sensors

These sensors are acting as ground truth as the purpleair-sensors are known to have quality control issues. Data is querried from the Airnow API and saved as .csv file.

For more detail: https://docs.airnowapi.org/Data/docs#sampleUsage


In [18]:
#Collect the information
def main():
    # API parameters
    options = {}
    options["url"] = "https://www.airnowapi.org/aq/data/"
    options["start_date"] = "2020-04-06"
    options["start_hour_utc"] = "00"
    options['end_date'] = '2020-04-07'
    options['end_hour_utc'] = "17"
    options["parameters"] = "OZONE,PM25,PM10"
    options["bbox"] = "-113.989989,37.043937,-108.936278,41.989859" #minx,miny,maxx,maxy for state of utah (includes wyoming corner)
    options["data_type"] = "B"
    options["format"] = "text/csv"
    options["api_key"] = "0ABAAB64-E208-4B1A-B6D7-735E98877CE0"

    # API request URL
    REQUEST_URL = options["url"] \
                  + "?startDate=" + options["start_date"] \
                  + "T"+options['start_hour_utc']\
                  + "&endDate=" + options["end_date"] \
                  + "T"+options['end_hour_utc']\
                  + "&parameters=" + options["parameters"] \
                  + "&BBOX=" + options["bbox"] \
                  + "&datatype=" + options["data_type"] \
                  + "&format=" + options["format"] \
                  + "&verbose=1" \
                  + "&API_KEY=" + options["api_key"]

    try:
        # Request AirNowAPI data
        print("Requesting AirNowAPI data...")

        # User's home directory.
        download_file_name = "AirNowAPI" + datetime.now().strftime("_%Y%M%d%H%M%S.csv")
        download_file = os.path.join('/home/boogie2/Data_Science/Purp_Air_Analysis', download_file_name)
        
        print(REQUEST_URL)
        # Perform the AirNow API data request

        
        urlretrieve(REQUEST_URL, download_file)

    except Exception as e:
        print("Unable perform AirNowAPI request")
        sys.exit(1)

#Execute Data Collection
if __name__ == "__main__":
    main()
    

Requesting AirNowAPI data...
https://www.airnowapi.org/aq/data/?startDate=2020-04-06T00&endDate=2020-04-07T17&parameters=OZONE,PM25,PM10&BBOX=-113.989989,37.043937,-108.936278,41.989859&datatype=B&format=text/csv&verbose=1&API_KEY=0ABAAB64-E208-4B1A-B6D7-735E98877CE0


In [19]:
column_names = ['Latitude',
           'Longitude',
          'Time',
          'Parameter',
          'Concentration',
          'Unit',
          'Raw_Concentration',
          'AQI',
          'Category',
          'Site_Name',
          'Site_Agency',
          'AQS_ID',
          'FULL_AQS_ID']

airnow_df = pd.read_csv('AirNowAPI_20205707135740.csv',header = None,names = column_names)

In [20]:
airnow_df.head()

,Latitude,Longitude,Time,Parameter,Concentration,Unit,Raw_Concentration,AQI,Category,Site_Name,Site_Agency,AQS_ID,FULL_AQS_ID
0,37.179000,-113.305200,2020-04-06T00:00,OZONE,45.0,PPB,42,1,Hurricane,Utah Department of Environmental Quality,490530007,840490530007,NaN
1,37.179000,-113.305200,2020-04-06T00:00,PM2.5,4.8,UG/M3,20,1,Hurricane,Utah Department of Environmental Quality,490530007,840490530007,NaN
2,37.198300,-113.150600,2020-04-06T00:00,OZONE,44.0,PPB,41,1,Zion NP,National Park Service,490530130,840490530130,NaN
3,37.747445,-113.055524,2020-04-06T00:00,OZONE,45.0,PPB,42,1,Enoch,Utah Department of Environmental Quality,490210005,840490210005,NaN
4,37.747445,-113.055524,2020-04-06T00:00,PM2.5,4.1,UG/M3,17,1,Enoch,Utah Department of Environmental Quality,490210005,840490210005,NaN


## Merge the Data:

Things to do:
1. Standardize the time 
    - Split Date and Hour Components
    - Append the existing datafiles from purple air
    - Select the data points which have lat long within the min max previously outlined

**PURPLE AIR DATA**

All data starts at 5:00AM on 04.07.2020 (MST)

In [21]:
#Create two directories for the two types of data
exp_dir = []
leg_dir = []

for entries in data_direct:
    #Collect the experimental data
    if "experimental" in str(entries):
        exp_dir.append(entries)
        
    #Collect the legacy data    
    if "legacy" in str(entries):
        leg_dir.append(entries)

print("The legacy directory has {0} entries".format(len(leg_dir)))
print("The experimental directory has {0} entries".format(len(exp_dir)))

The legacy directory has 14 entries
The experimental directory has 14 entries


In [22]:
#This function will help put the data into a larger dataframe
def concater(temp_dir):
    data = []

    for dirs in temp_dir:
        temp_df = pd.read_pickle(dirs)
        
        if 'legacy' in dirs:
            temp_df['Time']= dirs.split('legacy_')[1].split('.pkl')[0]
        
        if 'exp' in dirs:
            temp_df['Time']= dirs.split('experimental_')[1].split('.pkl')[0]
            
        data.append(temp_df)

    return(pd.concat(data))

In [23]:
legacy_df = concater(leg_dir)
exp_df = concater(exp_dir)

In [24]:
#Check the dimensions
print('Legacy length check: {0}'.
      format(len(legacy_df['Time'].drop_duplicates()) == len(leg_dir)))

print('Experimental length check: {0}'.
      format(len(exp_df['Time'].drop_duplicates()) == len(exp_dir)))

Legacy length check: True
Experimental length check: True


In [25]:
#Grab Utah sensors only:
-113.989989,37.043937,-108.936278,41.989859

ut_leg_df = legacy_df[(legacy_df['Lat']>=37.043937)&
          (legacy_df['Lat']<=41.98959) &
         (legacy_df['Lon']<= -108.936278) &
         (legacy_df['Lon']>= -113.989989)]

ut_exp_df = exp_df[(exp_df['Lat']>=37.043937)&
          (exp_df['Lat']<=41.98959) &
         (exp_df['Lon']<= -108.936278) &
         (exp_df['Lon']>= -113.989989)]

In [26]:
print("Utah Legacy Data Shape {0} x {1}".format(ut_leg_df.shape[0],
                                        ut_leg_df.shape[1]))

print("Utah Experimental Data Shape {0} x {1}".format(ut_exp_df.shape[0],
                                        ut_exp_df.shape[1]))

Utah Legacy Data Shape 14350 x 29
Utah Experimental Data Shape 7294 x 37


In [27]:
#Working in mountain time so will keep that: 
#seperate the time and date
ut_leg_df = ut_leg_df.reset_index(drop=True)
ut_leg_df['Time'] = ut_leg_df['Time'].apply(lambda x: 
                                       datetime.strptime(str('20_'+x),
                                                         "%y_%m_%d_%H_%M"))
ut_exp_df = ut_exp_df.reset_index(drop=True)
ut_exp_df['Time'] = ut_exp_df['Time'].apply(lambda x: 
                                       datetime.strptime(str('20_'+x),
                                                     "%y_%m_%d_%H_%M"))


In [28]:
#Set the timezone
tz = pytz.timezone("America/Denver")

In [29]:
ut_leg_df['Time'] = ut_leg_df['Time'].apply(lambda x: tz.localize(x))

ut_exp_df['Time'] = ut_exp_df['Time'].apply(lambda x: tz.localize(x))

**EPA Sensor Time Conversion**

In [30]:
airnow_df['Time'] = airnow_df['Time'].apply(lambda x: 
                                       datetime.strptime(str(x),
                                                     "%Y-%m-%dT%H:%M"))


In [31]:
#Convert from UTC
airnow_df['Time'] = airnow_df['Time'].apply(lambda x:
                                            pytz.utc.localize(x, is_dst=None).astimezone(tz))

So mountain standard time is six hours behind UTC.

Completed:
1. All the dataframes now have the same timezone.
2. Utah Data only selected (with a box for wyoming.

## Cleaning:

In [43]:
airnow_df.head()

,Latitude,Longitude,Time,Parameter,Concentration,Unit,Raw_Concentration,AQI,Category,Site_Name,Site_Agency,AQS_ID,FULL_AQS_ID
0,37.179000,-113.305200,2020-04-05 18:00:00-06:00,OZONE,45.0,PPB,42,1,Hurricane,Utah Department of Environmental Quality,490530007,840490530007,NaN
1,37.179000,-113.305200,2020-04-05 18:00:00-06:00,PM2.5,4.8,UG/M3,20,1,Hurricane,Utah Department of Environmental Quality,490530007,840490530007,NaN
2,37.198300,-113.150600,2020-04-05 18:00:00-06:00,OZONE,44.0,PPB,41,1,Zion NP,National Park Service,490530130,840490530130,NaN
3,37.747445,-113.055524,2020-04-05 18:00:00-06:00,OZONE,45.0,PPB,42,1,Enoch,Utah Department of Environmental Quality,490210005,840490210005,NaN
4,37.747445,-113.055524,2020-04-05 18:00:00-06:00,PM2.5,4.1,UG/M3,17,1,Enoch,Utah Department of Environmental Quality,490210005,840490210005,NaN


In [45]:
#How many sensors are registered:
print("There are {0} legacy sensors".format(len(ut_leg_df['ID'].drop_duplicates())))
print("There are {0} experimental sensors".format(len(ut_exp_df['ID'].drop_duplicates())))
print("There are {0} airnow (EPA) sensors".format(len(airnow_df['Category'].drop_duplicates())))

There are 1025 legacy sensors
There are 521 experimental sensors
There are 27 airnow (EPA) sensors


In [46]:
airnow_df['Category'].drop_duplicates()

0                      Hurricane
2                        Zion NP
3                          Enoch
5                           Erda
7                    Herriman #3
9                    Harrisville
10                     Rose Park
12                            NR
13                   Copper View
15                            BV
17                     Hawthorne
19                    Smithfield
20                        Lindon
22               Timpanogos Cave
23                  Spanish Fork
25                  Murphy Ridge
27                            P2
28                         Myton
29                      Roosvelt
31                    Whiterocks
32                Canyonlands NP
33                     Moxa Arch
35                         Ouray
36                     Vernal #4
38             James Town Mobile
41                       Redwash
42    Dinosaur National Monument
Name: Category, dtype: object